### imports

In [1]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

In [4]:
import jax.numpy as jnp
from jax import lax, jit
from jax.test_util import check_grads



### body

In [54]:
@jit
def scan_fun(x, y):
    # A loop using scan, note how I access y within the body
    n = x.shape[0]

    def body(carry, x):
        print(type(carry))
        # print(carry)
        curr, i = carry
        print('type(curr)', type(curr))
        print('type(i)', type(i))
        print('curr', curr)
        print('i', i)
        print('i+1', i+1)
        print('x[i]', x[i])
        print('curr + x[i]', curr + x[i])

        return (curr + x[i], i+1), None # next state, x? 

    init_curr = 10.
    init_i = 20
    (res, this_is_val), this_is_none = lax.scan(body, (init_curr, init_i), x) 
    # f = body, 
    # init = (0., 0), i.e. carry = 0., x = 0?, 
    # xs = None, length=None, reverse=False, unroll=1

    return (res, this_is_val), this_is_none
    return res# / n

@jit
def loop_fun(x, y):
    # A loop using fori_loop, note how I access both x and y within the body
    n = x.shape[0]

    def body(i, curr): 
        return curr + x[i]

    return lax.fori_loop(0, n, body, 0.)# / n



In [55]:
n = 50
m = 150
x = jnp.arange(n, dtype=jnp.float32)
y = jnp.arange(m, step=2., dtype=jnp.float32)

In [56]:
x

Array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
       39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49.],      dtype=float32)

In [57]:
jnp.sum(x)

Array(1225., dtype=float32)

In [58]:
scan_fun(x, y)

<class 'tuple'>
type(curr) <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>
type(i) <class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>
curr Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
i Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>
i+1 Traced<ShapedArray(int32[], weak_type=True)>with<DynamicJaxprTrace(level=2/0)>


IndexError: Too many indices for array: 1 non-None/Ellipsis indices for dim 0.

In [38]:

assert loop_fun(x, y) == scan_fun(x, y)

check_grads(scan_fun, (x, y), 1, eps=1e-2)  # All good
check_grads(loop_fun, (x, y), 1, eps=1e-2)  # raises ValueError: Reverse-mode differentiation does not work for lax.while_loop or lax.fori_loop. Try using lax.scan instead.


AssertionError: 